
# Extracción, Transformación y Carga (ETL)

El propósito de este notebook es extraer información de los archivos JSON, llevar a cabo procesos de limpieza y finalmente cargar los datos en formato CSV. Esto permitirá realizar, en etapas posteriores, el análisis exploratorio de datos, implementar funciones para los endpoints, y desarrollar modelos de machine learning.

## Pasos del Proceso ETL

1. **Extracción de Datos:** Obtendremos la información necesaria desde los archivos JSON, identificando las fuentes y recopilando los datos relevantes.

2. **Transformación de Datos:** Realizaremos operaciones de limpieza y transformación para garantizar la coherencia y calidad de los datos. Esto puede incluir la corrección de valores nulos, la normalización de formatos y la eliminación de duplicados.

3. **Carga de Datos:** Una vez que los datos estén preparados, los cargaremos en un formato estructurado, como CSV, que facilitará su análisis y utilización en las siguientes etapas del proyecto.



### Importación de Librerías

En esta sección, importaremos todas las bibliotecas necesarias en este notebook.

In [1]:
import pandas as pd 
import ast

### Funciones

Creación de las funciones que empleé para leer archivos JSON y desanidar las columnas en los conjuntos de datos `australian_user_review` y `australian_user_items`. Cabe mencionar que el archivo `output_steam_games` no presentaba columnas anidadas.

- **Función `cargar_json`**

  Esta función carga un archivo JSON y devuelve un DataFrame de pandas. Utiliza la función `open` para abrir el archivo en modo lectura con la codificación "MacRoman". Luego, itera sobre cada línea en el archivo, evalúa la línea como una expresión de Python usando `ast.literal_eval` y agrega las filas resultantes a una lista. Finalmente, crea y devuelve un DataFrame a partir de esa lista.

- **Función `expandir_columna_anidada`**

  La función `expandir_columna_anidada` toma un DataFrame de entrada y el nombre de una columna anidada. Su propósito es crear un nuevo DataFrame donde la columna anidada se ha expandido en filas individuales, combinando los datos de la columna anidada con los datos base de cada fila original.

In [2]:
def cargar_json(ruta_archivo):
    """
    Carga un archivo JSON y devuelve un DataFrame.

    Parameters:
        ruta_archivo (str): La ruta al archivo JSON.

    Returns:
        pd.DataFrame: El DataFrame cargado desde el archivo JSON.
    """
    filas = []

    # Abrir el archivo en modo lectura con la codificación "MacRoman"
    with open(ruta_archivo, encoding="MacRoman") as archivo:
        # Iterar sobre cada línea en el archivo
        for linea in archivo.readlines():
            # Evaluar la línea como una expresión de Python y agregarla a la lista
            filas.append(ast.literal_eval(linea))

    # Crear y devolver un DataFrame a partir de la lista de filas
    return pd.DataFrame(filas)


def expandir_columna_anidada(df_entrada, columna_anidada):
    """
    Expande una columna anidada en un DataFrame.

    Parameters:
    - df_entrada (pd.DataFrame): DataFrame de entrada con la columna anidada.
    - columna_anidada (str): Nombre de la columna anidada que se va a expandir.

    Returns:
    pd.DataFrame: Nuevo DataFrame con la columna anidada expandida.
    """
    # Inicializar una lista vacía para almacenar los registros expandidos.
    registros_expandidos = []

    # Iterar sobre cada fila del DataFrame de entrada.
    for indice, fila in df_entrada.iterrows():
        # Extraer los datos base de la fila eliminando la columna anidada y convirtiéndolos a un diccionario.
        datos_base = fila.drop(columna_anidada).to_dict()
        # Obtener la lista de datos anidados de la columna especificada.
        datos_anidados_lista = fila[columna_anidada]

        # Iterar sobre cada conjunto de datos anidados.
        for datos_anidados in datos_anidados_lista:
            # Crear un nuevo registro combinando los datos base y los datos anidados.
            nuevo_registro = {**datos_base, **datos_anidados}
            # Agregar el nuevo registro a la lista de registros_expandidos.
            registros_expandidos.append(nuevo_registro)

    # Crear un nuevo DataFrame a partir de la lista de registros_expandidos y devolverlo.
    df_expandido = pd.DataFrame(registros_expandidos)
    return df_expandido




## ETL del archivo `australian_user_reviews`

El objetivo de esta sección es realizar el proceso ETL específico para el archivo `australian_user_reviews`. Este conjunto de datos, que contiene las reseñas de usuarios a los juegos de la plataforma STEAM.


### Lectura del Archivo

Utilizamos las funciones `cargar_json` y `expandir_columna_anidada` para leer el archivo y procesar las columnas anidadas.


In [3]:
# Ruta del archivo JSON
ruta_json = "..\\Datsets\\australian_user_reviews.json"

# Cargar el archivo JSON en un DataFrame
user_reviews = cargar_json(ruta_json)

# Expandir los diccionarios en varias filas
user_reviews= expandir_columna_anidada(user_reviews, 'reviews')


In [4]:
user_reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


### Valores nulos

El dataframe user_reviews no contiene valores nulos

In [5]:
user_reviews.isnull().sum()

user_id        0
user_url       0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64

### Duplicados
Eliminamos los valores duplicados.

In [6]:
user_reviews.drop_duplicates(inplace=True)
user_reviews.shape

(58431, 9)

### Eliminación de Columnas

Eliminaré las columnas "user_url" y "helpful", ya que no son relevantes para el proyecto. Además, la columna "funny" y la columna "last_edited" contienen muchos datos en blanco, por lo que también serán eliminadas.

In [7]:
#Eliminamos la columna user_reviews y la columna helpful
user_reviews = user_reviews.drop('user_url', axis=1)
user_reviews = user_reviews.drop('helpful', axis=1)


Identificación de la cantidad de caracteres en blanco

In [8]:
# Identificar y contar las filas que contienen solo espacios o caracteres en blanco en la columna 'funny'
filas_espacios_funny = user_reviews[user_reviews['funny'].apply(lambda x: x.isspace() or not x)].index

# Imprimir el resultado
print("Número de filas que contienen caracteres en blanco en la columna 'funny':", len(filas_espacios_funny))

# Identificar y contar las filas que contienen solo espacios o caracteres en blanco en la columna 'last_edited'
filas_espacios_last = user_reviews[user_reviews['last_edited'].apply(lambda x: x.isspace() or not x)].index

# Imprimir el resultado
print("Número de filas que contienen caracteres en blanco en la columna 'last_edited':", len(filas_espacios_last))

Número de filas que contienen caracteres en blanco en la columna 'funny': 50421
Número de filas que contienen caracteres en blanco en la columna 'last_edited': 52394


In [9]:
#Eliminamos la columa funny
user_reviews = user_reviews.drop('funny', axis=1)
#Eliminamos la columa last_edited
user_reviews = user_reviews.drop('last_edited', axis=1)


### Transformación de la columna "Posted" a formato de fecha 

La columna "Posted" hace referencia a la fecha en que se realizó la reseña. Contiene la palabra "Posted" seguida de la fecha. Para mejorar la estructura, procederemos a eliminar la palabra "Posted" y convertiremos el resto a formato de fecha. También extraeremos el año y crearemos la columna "posted_year". Posteriormente, eliminaremos la columna original "posted".

In [10]:
#Quitamos la palabra Posted 
user_reviews['posted'] = user_reviews['posted'].str.replace('Posted ', '')
#Convertimos a fecha
user_reviews['posted'] = pd.to_datetime(user_reviews['posted'], errors='coerce')
#Extraemos el año y creamos otra columna llamada posted_year
user_reviews['posted_year'] = user_reviews['posted'].dt.year
#Borramos la columna posted
user_reviews=user_reviews.drop('posted',axis=1)

## ETL del archivo australian_users_items

El objetivo de esta sección es realizar el proceso ETL específico para el archivo `australian_users_items`. Este conjunto de datos, contiene la información sobre los juegos que tienen los usuarios de STEAM.

In [11]:
# Ruta del archivo JSON
ruta_json = "..\\Datsets\\australian_users_items.json"

# Cargar el archivo JSON en un DataFrame
user_items = cargar_json(ruta_json)

# Expandir los diccionarios en varias filas
user_items = expandir_columna_anidada(user_items, "items")

In [12]:
user_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


### Valores nulos

El dataframe user_items no contiene valores nulos

In [13]:
user_items.isnull().sum()

user_id             0
items_count         0
steam_id            0
user_url            0
item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

### Duplicados
Eliminamos los valores duplicados.

In [14]:
user_items.drop_duplicates(inplace=True)
user_items.shape

(5094105, 8)

### Eliminación de Columnas

Eliminaré la columna "user_url" y 'playtime_2weeks', ya que no son relevantes para este proyecto. La columan steam_id tambien se va a borrar porque la informacion que proporciona es similar a la de user_id

In [15]:
#Eliminamos la columa user_url
user_items = user_items.drop('user_url', axis=1)
user_items = user_items.drop('playtime_2weeks', axis=1)
user_items = user_items.drop('steam_id', axis=1)

## ETL del archivo output_steam_games

El objetivo de esta sección es realizar el proceso ETL específico para el archivo `output_steam_game`. Este conjunto de datos, que contiene la información sobre los juegos

In [16]:
# Ruta del archivo JSON
ruta_json = "..\\Datsets\\output_steam_games.json"

# Cargar el archivo JSON en un DataFrame
steam_games=pd.read_json(ruta_json, lines=True)

In [17]:
steam_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


### Valores nulos

El DataFrame steam_games contiene muchos valores nulos.

In [18]:
steam_games.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Primero, vamos a eliminar todas las filas en las que todos sus valores sean nulos.

In [19]:
#Borramos las filas en las que todos sus valores sean nulos
steam_games.dropna(how='all', inplace=True)
#Contamos cuantos valores nulos hay por columna
steam_games.isnull().sum()


publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

En el caso de los géneros, extraeré una lista de los géneros existentes. Posteriormente, completaré los géneros faltantes utilizando la información de la columna "tag". Sin embargo, se limitará la inclusión de géneros a aquellos que estén presentes en la lista original de géneros, ya que la columna "tags" contiene más categorías que la columna de géneros.

In [20]:
# Convertimos la columna 'genres' a tipo lista, conservando NaN
steam_games['genres'] = steam_games['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Expandimos las listas en filas
steam_games_genres = steam_games.explode('genres')
# Obtener la lista de géneros únicos, incluyendo los valores nulos
lista_generos = steam_games_genres['genres'].unique()
steam_games['tags'] = steam_games['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
steam_games['genres'] = steam_games.apply(
    lambda row: [tag for tag in row['tags'] if tag in lista_generos] if isinstance(row['tags'], list) else row['genres'],
    axis=1
)

En el caso de la columna "Developers", seguiré una estrategia similar. Inicialmente, extraeré una lista de los desarrolladores únicos presentes en los datos. Luego, para aquellas filas donde la columna "Developers" está vacía pero el "Publisher" se encuentra en la lista de desarrolladores, asumiremos que la compañía que publicó el juego es la misma que lo desarrolló.

In [21]:
#Lista de desarolladoras
lista_desarroladores =steam_games['developer'].unique()


for index, row in steam_games.iterrows():
    # Verificar si el valor en 'developer' está vacío y si 'publisher' está en la lista de desarrolladores
    if pd.isnull(row['developer']) and row['publisher'] in lista_desarroladores:
        # Rellenar 'developer' con el valor de 'publisher'
        steam_games.at[index, 'developer'] = row['publisher']


### Duplicados

Revisamos cuántas filas tienen valores duplicados.


In [25]:
# Verificamos filas duplicadas en el dataframe
duplicados = steam_games[steam_games.duplicated(['publisher', 'app_name', 'title', 'url', 
                                                                 'release_date', 'reviews_url', 'price', 'early_access', 
                                                                 'id', 'developer'])]
print(f"Cantidad de filas duplicadas: {len(duplicados)}")
subset_cols = ['publisher', 'app_name', 'title', 'url', 'release_date', 'reviews_url', 'price', 'early_access', 'id', 'developer']
steam_games.drop_duplicates(subset=subset_cols, inplace=True)


Cantidad de filas duplicadas: 0


### Eliminación de Columnas

En esta etapa, procedemos a eliminar las columnas `reviews_url` y `url` debido a su poca relevancia para este proyecto. Asimismo, decidimos eliminar la columna `publisher` debido a la presencia de numerosos valores nulos. Dado que las columnas `app_name` y `title` contienen información similar, y la columna `title` tiene 2050 valores nulos, optamos por conservar la columna `app_name`.


In [26]:
steam_games = steam_games.drop('reviews_url', axis=1)
steam_games = steam_games.drop('url', axis=1)
steam_games = steam_games.drop('publisher', axis=1)
steam_games = steam_games.drop('title', axis=1)

### Transformación de la Columna "release_date"

La columna "release_date" se transformará a tipo de dato fecha. Además, se extraerá el año y se creará la columna "release_year". Posteriormente, la columna original "release_date" será eliminada.

In [27]:
#Convertimos la columa release_date a formato fecha
steam_games['release_date']=pd.to_datetime(steam_games['release_date'], errors='coerce', exact=False)
# Extraer el año y crear una nueva columna 'release_year'
steam_games['release_year'] = steam_games['release_date'].dt.year
# Eliminar la columna 'release_date'
steam_games = steam_games.drop(columns=['release_date'])

### Transformación de la columna "price"

La columna "price" se transformará a tipo numérico. Dado que contiene texto que indica que el juego es gratuito, se reemplazará la cadena de texto por el valor numérico 0.


In [28]:
steam_games['price'] = steam_games['price'].replace('Free To Play', 0)
steam_games['price'] = steam_games['price'].replace('Free to Play', 0)
steam_games['price'] = steam_games['price'].replace('Free', 0)
steam_games['price'] = steam_games['price'].replace('None', 0)
steam_games['price'] = steam_games['price'].replace('Free HITMAN™ Holiday Pack', 0)
steam_games['price'] = steam_games['price'].replace('Free Demo', 0)
steam_games['price'] = steam_games['price'].replace('Play for Free!', 0)
steam_games['price'] = steam_games['price'].replace('Play WARMACHINE: Tactics Demo', 0)
steam_games['price'] = steam_games['price'].replace('Install Now', 0)
steam_games['price'] = steam_games['price'].replace('Install Theme', 0)
steam_games['price'] = steam_games['price'].replace('Third-party', 0)
steam_games['price'] = steam_games['price'].replace('Play Now', 0)
steam_games['price'] = steam_games['price'].replace('Play the Demo', 0)
steam_games['price'] = steam_games['price'].replace('Free to Try', 0)
steam_games['price'] = steam_games['price'].replace('Free to Use', 0)
steam_games['price'] = steam_games['price'].replace('Free Movie', 0)
steam_games['price'] = steam_games['price'].replace('Free Mod', 0)
steam_games['price'] = steam_games['price'].replace('Free Mod', 0)
steam_games['price'] = steam_games['price'].replace('Starting at $499.00', 499)
steam_games['price'] = steam_games['price'].replace('Starting at $449.00', 449)


## Convertir los DataFrames a CSV


In [29]:
# Convertir el DataFrame 'steam_games' a CSV
steam_games.to_csv("..\\Datsets\\CSV\\steam_games.csv", index=False)

# Convertir el DataFrame 'user_reviews' a CSV
user_reviews.to_csv("..\\Datsets\\CSV\\user_reviews.csv", index=False)

# Convertir el DataFrame 'user_items' a CSV
user_items.to_csv("..\\Datsets\\CSV\\user_items.csv", index=False)
